In [57]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [17]:
def getByUrl(url):
    soup = BeautifulSoup(requests.get(url).text, 'html.parser')
    listContainer = soup.find(id='listContainer')
    titles = listContainer.findAll('header')
    results = []
    for title in titles:
        results.append((title.a.text, title.a['href']))
    return results

In [23]:
all_links = []
for page in range(1, 10):
    url = 'https://xpir.ru/finsources?page=' + str(page) + '&finSourceType=08d75e615e7d46fd80bc6e0d9fc9606f&auditory=b4db5577b2f94d38be9f6eca89f71dc4'
    all_links.extend(getByUrl(url))

In [80]:
def getConcrete(name, link):
    url = 'https://xpir.ru' + link
    soup = BeautifulSoup(requests.get(url).text, 'html.parser')
    
    link = soup.find('div', {"class": "fin-link"}).a['href']
    volumes = soup.findAll('span', {"class": "fin-investment-size-text"})
    volume_low, volume_high = volumes[0].text if len(volumes) > 0 else None, volumes[1].text if len(volumes) > 1 else None
    raw_data = soup.find('div', {'class': 'fin-not-idividual'}).findAll('div', {'class': 'info-block-in'})
    description = soup.find('div', {'class': 'ei-content-text expandable-description'}).text
    text = soup.find('div', {'class': 'expandable-part'}).text
    data = [data.text.strip() for data in raw_data]
    vidy_poderzhki = data[1][len('Виды поддержки'):] if data[1].startswith(u'Виды поддержки') else None
    stadii_rabot = data[3][len('Стадии работ'):] if data[3].startswith(u'Стадии работ') else (data[1][len('Стадии работ'):] if data[1].startswith(u'Стадии работ') else None)
    tip_isto4nika = data[5][len('Тип источника поддержки'):] if len(data) > 5 and data[5].startswith(u'Тип источника поддержки') else (data[3][len('Тип источника поддержки'):] if data[3].startswith(u'Тип источника поддержки') else None)
    
    return {'name': name, 'link': link, 'description': description, 'text': text, 'volume_low': volume_low, 'volume_high': volume_high, 
            'Виды поддержки': vidy_poderzhki, "Стадии работ": stadii_rabot, "Тип источника поддержки": tip_isto4nika}

In [81]:
dataset = []
for name, link in all_links:
    data = getConcrete(name, link)
    dataset.append(data)

In [82]:
df = pd.DataFrame(dataset)
for col in df.columns:
    df[col] = df[col].str.strip()
df

,description,link,name,text,volume_high,volume_low,Виды поддержки,Стадии работ,Тип источника поддержки
0,Фонд «Сколково» – фонд развития Центра разрабо...,http://sk.ru/foundation/,Фонд «Сколково»,Фонд «Сколково» – фонд развития Центра разрабо...,None,Р300 000 000,"Грант, Инфраструктура","Производство, Создание прототипа","Кластер, Фонд"
1,Фонд Егора Гайдара - некоммерческая организаци...,http://gaidarfund.ru,Фонд Егора Гайдара,Фонд Егора Гайдара - некоммерческая организаци...,Р100 000,Р10 000,"Грант, Обучение, Премия, Стажировка, Стипендия","Прикладные исследования, Фундаментальные иссле...",Фонд
2,ОАО «Российская венчурная компания» — государс...,http://www.rusventure.ru,"ОАО ""Российская венчурная компания""",ОАО «Российская венчурная компания» — государс...,None,Р3 000 000,"Венчурная, Госзаказ, Информационная, Кадровая","Прикладные исследования, Производство, Создани...",Фонд
3,Российский гуманитарный научный фонд - самоупр...,http://www.rfh.ru,Российский гуманитарный научный фонд,Российский гуманитарный научный фонд - самоупр...,None,Р500 000,Грант,"Прикладные исследования, Создание прототипа, Ф...",Фонд
4,Российский фонд фундаментальных исследований (...,http://www.rfbr.ru/,РФФИ (Российский фонд фундаментальных исследов...,Российский фонд фундаментальных исследований (...,None,Р400 000 000,Грант,Фундаментальные исследования,Фонд
5,Фонд перспективных исследований создан для сод...,http://www.fpi.gov.ru,Фонд перспективных исследований,Фонд перспективных исследований создан для сод...,None,None,Госзаказ,"Прикладные исследования, Создание прототипа, Ф...",Фонд
6,Фонд содействия развитию малых форм предприяти...,http://www.fasie.ru/,Фонд содействия развитию малых форм предприяти...,Фонд содействия развитию малых форм предприяти...,Р15 000 000,Р200 000,Грант,"Прикладные исследования, Производство, Создани...",Фонд
7,РФТР – ключевой государственный институт по фи...,http://www.rftr.ru/,Российский фонд технологического развития,РФТР – ключевой государственный институт по фи...,None,Р150 000 000,Кредит,"Прикладные исследования, Производство, Создани...",Фонд
8,Фонд инфраструктурных и образовательных програ...,http://www.rusnano.com,Фонд инфраструктурных и образовательных программ,Фонд инфраструктурных и образовательных програ...,None,Р1 000 000,"Венчурная, Кредит, Субсидия","Производство, Создание прототипа",Фонд
9,Немецкое научно-исследовательское общество (DE...,http://www.dfg.de/,German Research Foundation (DFG),Немецкое научно-исследовательское общество (DE...,None,€10 000,"Грант, Стипендия","Прикладные исследования, Создание прототипа, Ф...","Объединение, Фонд"


In [83]:
df = df[["name", "link", "description", "text", "volume_low", "volume_high", "Виды поддержки", "Стадии работ", "Тип источника поддержки"]]
df

,name,link,description,text,volume_low,volume_high,Виды поддержки,Стадии работ,Тип источника поддержки
0,Фонд «Сколково»,http://sk.ru/foundation/,Фонд «Сколково» – фонд развития Центра разрабо...,Фонд «Сколково» – фонд развития Центра разрабо...,Р300 000 000,None,"Грант, Инфраструктура","Производство, Создание прототипа","Кластер, Фонд"
1,Фонд Егора Гайдара,http://gaidarfund.ru,Фонд Егора Гайдара - некоммерческая организаци...,Фонд Егора Гайдара - некоммерческая организаци...,Р10 000,Р100 000,"Грант, Обучение, Премия, Стажировка, Стипендия","Прикладные исследования, Фундаментальные иссле...",Фонд
2,"ОАО ""Российская венчурная компания""",http://www.rusventure.ru,ОАО «Российская венчурная компания» — государс...,ОАО «Российская венчурная компания» — государс...,Р3 000 000,None,"Венчурная, Госзаказ, Информационная, Кадровая","Прикладные исследования, Производство, Создани...",Фонд
3,Российский гуманитарный научный фонд,http://www.rfh.ru,Российский гуманитарный научный фонд - самоупр...,Российский гуманитарный научный фонд - самоупр...,Р500 000,None,Грант,"Прикладные исследования, Создание прототипа, Ф...",Фонд
4,РФФИ (Российский фонд фундаментальных исследов...,http://www.rfbr.ru/,Российский фонд фундаментальных исследований (...,Российский фонд фундаментальных исследований (...,Р400 000 000,None,Грант,Фундаментальные исследования,Фонд
5,Фонд перспективных исследований,http://www.fpi.gov.ru,Фонд перспективных исследований создан для сод...,Фонд перспективных исследований создан для сод...,None,None,Госзаказ,"Прикладные исследования, Создание прототипа, Ф...",Фонд
6,Фонд содействия развитию малых форм предприяти...,http://www.fasie.ru/,Фонд содействия развитию малых форм предприяти...,Фонд содействия развитию малых форм предприяти...,Р200 000,Р15 000 000,Грант,"Прикладные исследования, Производство, Создани...",Фонд
7,Российский фонд технологического развития,http://www.rftr.ru/,РФТР – ключевой государственный институт по фи...,РФТР – ключевой государственный институт по фи...,Р150 000 000,None,Кредит,"Прикладные исследования, Производство, Создани...",Фонд
8,Фонд инфраструктурных и образовательных программ,http://www.rusnano.com,Фонд инфраструктурных и образовательных програ...,Фонд инфраструктурных и образовательных програ...,Р1 000 000,None,"Венчурная, Кредит, Субсидия","Производство, Создание прототипа",Фонд
9,German Research Foundation (DFG),http://www.dfg.de/,Немецкое научно-исследовательское общество (DE...,Немецкое научно-исследовательское общество (DE...,€10 000,None,"Грант, Стипендия","Прикладные исследования, Создание прототипа, Ф...","Объединение, Фонд"


In [90]:
for col in df.columns:
    df[col] = df[col].apply(lambda x: unicode(x))

In [95]:
import sys  

reload(sys)  
sys.setdefaultencoding('utf8')

In [96]:
df.to_excel("xpir.xlsx", encoding="UTF-8", engine='xlsxwriter')